# Mesh Refinement Tutorial - Cantilevered Hyperelastic Beam

This tutorial is a slight modification of the one found at https://jsdokken.com/dolfinx-tutorial/chapter2/hyperelasticity.html, covering the effects of mesh refinement on the simulation of a cantilevered hyperelastic beam's deflection.

First, we import dependencies and setup the simulation domain.

In [15]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

[2025-04-26 09:41:09.065] [info] Extract basic topology: 4000->4000
[2025-04-26 09:41:09.065] [info] Build local dual graph
[2025-04-26 09:41:09.065] [info] Build local part of mesh dual graph (mixed)
[2025-04-26 09:41:09.065] [info] GPS pseudo-diameter:(28) 499-0
[2025-04-26 09:41:09.065] [info] Create topology (single cell type)
[2025-04-26 09:41:09.065] [info] Create topology (generalised)
[2025-04-26 09:41:09.065] [info] Computing communication graph edges (using NBX algorithm). Number of input edges: 1
[2025-04-26 09:41:09.065] [info] Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
[2025-04-26 09:41:09.066] [info] Compute ghost indices
[2025-04-26 09:41:09.066] [info] Computing communication graph edges (using PCX algorithm). Number of input edges: 0
[2025-04-26 09:41:09.066] [info] Finished graph edge discovery using PCX algorithm. Number of discovered edges 0
[2025-04-26 09:41:09.066] [info] Computing communication graph edges (using NBX algorithm

To simplify boundary condition application, we next create functions to select the left and right groups of facets.

In [16]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

[2025-04-26 09:41:09.098] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 09:41:09.098] [info] Computing mesh connectivity 2-3 from transpose.
[2025-04-26 09:41:09.098] [info] Requesting connectivity (2, 0) - (0, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (2, 0) - (2, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (0, 0) - (3, 0)
[2025-04-26 09:41:09.098] [info] Computing mesh connectivity 0-3 from transpose.
[2025-04-26 09:41:09.098] [info] Requesting connectivity (3, 0) - (0, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (2, 0) - (0, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (2, 0) - (2, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (0, 0) - (3, 0)
[2025-04-26 09:41:09.098] [info] Requesting connectivity (3, 0) - (0, 0)


We can then apply the left side boundary conditions (fixed):

In [17]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force and traction, as well as test and solution functions.

In [18]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

v = ufl.TestFunction(V)
u = fem.Function(V)

Now we specify the kinematic properties:

In [19]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

...as well the hyperelasticity model parameters:

In [20]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

Define the weak form with traction integral over all facets with value 2.
 We set the quadrature degree for the integrals to 4, and use Dolfinx's nonlinearproblem solver.

In [21]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

problem = NonlinearProblem(F, u, bcs)

[2025-04-26 09:41:09.138] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 09:41:09.138] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-04-26 09:41:09.138] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 09:41:09.138] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-04-26 09:41:09.143] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 09:41:09.143] [info] Requesting connectivity (3, 0) - (2, 0)


Next we customize a newton solver:

In [22]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

[2025-04-26 09:41:09.155] [info] Column ghost size increased from 0 to 0


...and create a function to plot the deformation results at each time step:

In [23]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("b-deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

[2025-04-26 09:41:12.205] [info] Checking required entities per dimension
[2025-04-26 09:41:12.205] [info] Cell type: 0 dofmap: 500x27
[2025-04-26 09:41:12.205] [info] Global index computation
[2025-04-26 09:41:12.205] [info] Got 4 index_maps
[2025-04-26 09:41:12.205] [info] Get global indices


Finally, we solve the problem over several time steps, updating the z-component of the traction.

In [24]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

[2025-04-26 09:41:12.538] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 09:41:15.618] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 09:41:18.125] [info] Newton iteration 2: r (abs) = 22.245488797060162 (tol = 1e-08), r (rel) = 0.13427794209082689 (tol = 1e-08)
[2025-04-26 09:41:18.430] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 09:41:20.935] [info] Newton iteration 3: r (abs) = 2.4326133716237943 (tol = 1e-08), r (rel) = 0.014683710500775273 (tol = 1e-08)
[2025-04-26 09:41:21.245] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 09:41:23.760] [info] Newton iteration 4: r (abs) = 4.431578544918353 (tol = 1e-08), r (rel) = 0.02674983915409122 (tol = 1e-08)
[2025-04-26 09:41:24.066] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 09:41:26.570] [info] Newton iteration 5: r (abs) = 0.14418915603114846 (tol = 1e-08), r (rel) = 0.0008703527856953385 (tol = 1e-08)
[2025-04-26 09:41:26.874] [info] PETSc

<img src="b-deformation.png" alt="Beam Deformation" width="500"/>